O primeiro algoritmo de roteamento, chamado Maze Router, foi proposto por
Lee no ano de 1961. Este algoritmo é ótimo, em relação ao comprimento do fio, mas possui um elevado custo em tempo de processamento e consumo de memória.

O Algoritmo de Lee é um algoritmo de roteamento usado para encontrar a rota mais curta de um ponto a outro em um labirinto. Em um projeto de VLSI e FPGA, o algoritmo de Lee garante encontrar o caminho mais curto para uma única rede, se existir, mas o custo desse algoritmo é bastante alto.



Vantagens
* Garantia de caminho mínimo
* Garantia de encontrar conexão entre 2 terminais, se existir.

Desvantagens: 
* Lento
* Requer grande memória para layout denso


#Algoritmo

In [ ]:
import numpy
from random import randint
from pylab import *

## Gerando uma matriz 4x4

In [ ]:
map_grid = np.random.randint(2, size=(40, 40))
#w = 40
#h = 40
'''map_grid = np.full((w, h), int(5), dtype=np.int8)
map_grid[3, 3:8] = 0
map_grid[3:10, 7] = 0
map_grid[10, 3:8] = 0
map_grid[17, 13:17] = 0
map_grid[10:17, 13] = 0
map_grid[10, 13:17] = 0
map_grid[5, 2] = 10
map_grid[15, 15] = 7'''
print(map_grid)
w = len(map_grid)
h = len(map_grid)


[[1 1 0 ... 1 1 0]
 [0 1 0 ... 1 1 1]
 [0 0 0 ... 1 1 0]
 ...
 [1 0 1 ... 1 1 0]
 [0 1 1 ... 0 1 0]
 [0 0 0 ... 1 0 0]]


In [ ]:
class Lee(object):

    def __init__(self, start, goal):
        self.closed_0 = []
        self.closed_1 = []
        self.open_0 = [start]
        self.open_1 = [goal]
        self.path_0 = []
        self.path_1 = []

    def surround(self, node, open, closed, num):
        l = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for i in l:
            if node[0] + i[0] < 0 or node[1] + i[1] < 0 or node[0] + i[0] > w - 1 or node[1] + i[1] > w - 1:
                continue
            if map_grid[node[0] + i[0]][node[1] + i[1]] == 0:  
                continue
            for n in closed:
                if node[0] + i[0] == n[0] and node[1] + i[1] == n[1]:
                    break
            for m in open:
                if node[0] + i[0] == m[0] and node[1] + i[1] == m[1]:
                    break
            else:
                open.append((node[0] + i[0], node[1] + i[1], num))

    def search(self):
        iter = 0
        while True:
            if iter == 3:
                iter = 0
            iter += 1
            open_0 = self.open_0       
            self.closed_0 += open_0    
            self.open_0 = []          
            for n in open_0:
                self.surround(n, self.open_0, self.closed_0, iter)
                for i in self.open_0:       
                    for j in self.open_1:
                        if i[0] == j[0] and i[1] == j[1]:
                            return i
            print('0', self.open_0)
            open_1 = self.open_1      
            self.closed_1 += open_1
            self.open_1 = []
            for n in open_1:
                self.surround(n, self.open_1, self.closed_1, iter)
                for i in self.open_1: 
                    for j in self.open_0:
                        if i[0] == j[0] and i[1] == j[1]:
                            return i
            print('1', self.open_1)


    def path(self):
        meet_pin_0 = self.search()
        meet_pin_1 = meet_pin_0
        self.path_1.append(meet_pin_0)
        print(meet_pin_0)
        self.closed_0.reverse()
        self.closed_1.reverse()
        l = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        flag_0 = 1
        for i in self.closed_0:
            if flag_0:
                for d in l:
                    if meet_pin_0[0] + d[0] == i[0] and meet_pin_0[1] + d[1] == i[1]:
                        self.path_0.append(i)
                        meet_pin_0 = i
                        flag_0 = 0
                        print(i)
            else:
                for d in l:
                    if meet_pin_0[0] + d[0] == i[0] and meet_pin_0[1] + d[1] == i[1]:
                        if meet_pin_0[2] == 1 and i[2] == 3:
                            self.path_0.append(i)
                            meet_pin_0 = i
                        elif meet_pin_0[2] - 1 == i[2]:
                            self.path_0.append(i)
                            meet_pin_0 = i
        flag_1 = 1
        for i in self.closed_1:
            if flag_1:
                for d in l:
                    if meet_pin_1[0] + d[0] == i[0] and meet_pin_1[1] + d[1] == i[1]:
                        self.path_1.append(i)
                        meet_pin_1 = i
                        flag_1 = 0
                        print(i)
            else:
                for d in l:
                    if meet_pin_1[0] + d[0] == i[0] and meet_pin_1[1] + d[1] == i[1]:
                        if meet_pin_1[2] == 1 and i[2] == 3:
                            self.path_1.append(i)
                            meet_pin_1 = i
                        elif meet_pin_1[2] - 1 == i[2]:
                            self.path_1.append(i)
                            meet_pin_1 = i

    def draw_path(self):
        for i in self.closed_0:
            map_grid[int(i[0]), int(i[1])] = 8
        for i in self.closed_1:
            map_grid[int(i[0]), int(i[1])] = 2
        for i in self.path_0:
            map_grid[int(i[0]), int(i[1])] = 4
        for i in self.path_1:
            map_grid[int(i[0]), int(i[1])] = 4
        plt.imshow(map_grid, cmap=plt.cm.hot, interpolation='nearest', vmin=0, vmax=10)
        plt.colorbar()
        xlim(-1, 20) 
        ylim(-1, 20)  
        my_x_ticks = numpy.arange(0, 20, 1)
        my_y_ticks = numpy.arange(0, 20, 1)
        plt.xticks(my_x_ticks)
        plt.yticks(my_y_ticks)
        plt.grid(True)
        plt.show()

start = (5, 2, 0)
goal = (15, 15, 0)
lee = Lee(start, goal)
lee.path()
lee.draw_path()

A saída de streaming foi truncada nas últimas 5000 linhas.
1 [(11, 16, 1), (13, 16, 1), (12, 15, 1), (12, 17, 1), (10, 17, 1), (11, 18, 1), (15, 16, 1), (14, 17, 1), (8, 17, 1), (9, 18, 1), (6, 17, 1), (7, 18, 1), (8, 15, 1)]
0 [(4, 1, 2), (3, 0, 2), (4, 3, 2)]
1 [(12, 16, 2), (11, 17, 2), (14, 16, 2), (9, 17, 2), (7, 17, 2), (8, 16, 2), (7, 15, 2), (9, 15, 2)]
0 [(3, 1, 3), (4, 2, 3)]
1 [(11, 16, 3), (13, 16, 3), (12, 15, 3), (12, 17, 3), (10, 17, 3), (11, 18, 3), (15, 16, 3), (14, 17, 3), (8, 17, 3), (9, 18, 3), (6, 17, 3), (7, 18, 3), (8, 15, 3)]
0 [(4, 1, 1), (3, 0, 1), (4, 3, 1)]
1 [(12, 16, 1), (11, 17, 1), (14, 16, 1), (9, 17, 1), (7, 17, 1), (8, 16, 1), (7, 15, 1), (9, 15, 1)]
0 [(3, 1, 2), (4, 2, 2)]
1 [(11, 16, 2), (13, 16, 2), (12, 15, 2), (12, 17, 2), (10, 17, 2), (11, 18, 2), (15, 16, 2), (14, 17, 2), (8, 17, 2), (9, 18, 2), (6, 17, 2), (7, 18, 2), (8, 15, 2)]
0 [(4, 1, 3), (3, 0, 3), (4, 3, 3)]
1 [(12, 16, 3), (11, 17, 3), (14, 16, 3), (9, 17, 3), (7, 17, 3), (8, 16, 3), 

KeyboardInterrupt: ignored